# MACHINE LEARNING - GRID SEARCH 1

In this notebook we do the very first optimization trial to the classification problem, composed of the following steps:

- Data cleaning: in this case, the only operation is just dropping the Lomb-Scargle (Periodic) features from the featurized datasets (synthetic dataset DS4 and real dataset).
- Training/test set and validation set split.
- Model optimization and training: we optimize and train a `HistGradientBoostingClassifier` on the training/test set, using telescopic search, crossvalidation, and based on the AUC metrics.
- Model validation: we apply the fitted model to the validation set, to quantify the performance of our model on new data. We use the metrics precision, accuracy, recall and F1 score. We also do a threshold inspection to seek for a apropriate threshold.
- Model application: we apply the model to the real dataset, classifying the stars in the real dataset as pulsating or non-pulsating.

## Modules and configuration

### Modules

In [30]:
import warnings

import math
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score, f1_score, \
    RocCurveDisplay


from sklearn.experimental import enable_hist_gradient_boosting
# Not sure why this 'experimental' import is needed, as the sklearn version is 0.24.x > 0.21.x
from sklearn.ensemble import HistGradientBoostingClassifier

import matplotlib.pyplot as plt

### Configuration

In [2]:
# CONFIGURATION:
RANDOM_STATE = 11 # For reproducibility

# FILES AND FOLDERS
SYNTH_DATASET = "../data/DATASETS_CESIUM/cesium_DS4_Dataset.csv"
REAL_DATASET = "../data/DATASETS_CESIUM/cesium_GTO_Dataset.csv"
OUT_FOLDER = "../data/DATASETS_ML/"
SYNTH_PROCESSED_DS = "DS4_Dataset.csv"
REAL_PROCESSED_DS = "GTO_Dataset.csv"

# FEATURES TO KEEP - To remove the Lomb-Scargle (Periodic) 'cesium' features
TARGET_DS4 = ['Pulsating']
KEEP_CS_FEATURES = ['all_times_nhist_numpeaks',
                   'all_times_nhist_peak1_bin', 'all_times_nhist_peak2_bin', 'all_times_nhist_peak3_bin', 'all_times_nhist_peak4_bin',
                   'all_times_nhist_peak_1_to_2', 'all_times_nhist_peak_1_to_3', 'all_times_nhist_peak_1_to_4',
                   'all_times_nhist_peak_2_to_3', 'all_times_nhist_peak_2_to_4',
                   'all_times_nhist_peak_3_to_4',
                   'all_times_nhist_peak_val',
                   'avg_double_to_single_step', 'avg_err', 'avgt',
                   'cad_probs_1', 'cad_probs_10', 'cad_probs_20', 'cad_probs_30', 'cad_probs_40', 'cad_probs_50',
                   'cad_probs_100', 'cad_probs_500', 'cad_probs_1000', 'cad_probs_5000',
                   'cad_probs_10000', 'cad_probs_50000', 'cad_probs_100000', 'cad_probs_500000',
                   'cad_probs_1000000', 'cad_probs_5000000', 'cad_probs_10000000',
                   'cads_avg', 'cads_med', 'cads_std', 'mean',
                   'med_double_to_single_step', 'med_err',
                   'n_epochs', 'std_double_to_single_step', 'std_err',
                   'total_time', 'amplitude',
                   'flux_percentile_ratio_mid20', 'flux_percentile_ratio_mid35', 'flux_percentile_ratio_mid50',
                   'flux_percentile_ratio_mid65', 'flux_percentile_ratio_mid80',
                   'max_slope', 'maximum', 'median', 'median_absolute_deviation', 'minimum',
                   'percent_amplitude', 'percent_beyond_1_std', 'percent_close_to_median', 'percent_difference_flux_percentile',
                   'period_fast', 'qso_log_chi2_qsonu', 'qso_log_chi2nuNULL_chi2nu', 'skew', 'std',
                   'stetson_j', 'stetson_k', 'weighted_average', 'fold2P_slope_10percentile', 'fold2P_slope_90percentile']

# TRAIN/TEST SPLITS:
VALIDATION_SIZE = 0.25 # Fraction of DS4 reserved for model validation.

# MODEL OPTIMIZATION / TRAINING:
N_FOLD = 5 # n value for n-fold cross_validation
PARAM_GRID = { # Parameter grid, only valid for a 'HistGradientBoostingClassifier' outside a `Pipeline`.
    'learning_rate': [0.05, 0.1, 0.2],
    'max_iter': [50 ,100, 200], 
    'max_leaf_nodes': [None, 16, 31, 60],
    'max_depth': [None, 3, 12, 48],
    'min_samples_leaf': [10, 20, 40],
    'max_bins': [32, 128, 255]
}



### Functions

## Load and transform the synthetic dataset

In [3]:
ds4_orig = pd.read_csv(SYNTH_DATASET, sep=',', decimal='.')
ds4_orig.head()

,ID,Pulsating,frequency,amplitudeRV,offsetRV,refepochRV,phase,D3_samplingRV_idx,D3_PsRV_mean,D3_PsRV_median,...,freq_signif_ratio_31,freq_varrat,freq_y_offset,linear_trend,medperc90_2p_p,p2p_scatter_2praw,p2p_scatter_over_mad,p2p_scatter_pfold_over_mad,p2p_ssqr_diff_over_var,scatter_res_raw
0,RV-0,True,54.703173,1.133098,640.476258,2.457653e+06,0.737927,49,30.683447,9.072850,...,0.975457,1.362194e-05,-0.031323,-0.000023,1.019646,0.724440,1.144195,1.054973,2.207984,0.405728
1,RV-1,True,45.870515,0.945738,1392.825171,2.458556e+06,0.604099,223,104.231044,11.596005,...,1.075941,4.510759e-09,0.991227,-0.005244,21.598480,0.584204,1.245423,1.048418,2.904878,0.000910
2,RV-2,False,0.000000,0.000000,1799.268703,2.457479e+06,0.000000,336,83.770992,17.867630,...,1.054615,7.665963e-07,-0.278436,-0.000021,2.378014,0.525475,1.635725,1.185163,1.966647,0.141466
3,RV-3,False,0.000000,0.000000,918.007885,2.457612e+06,0.000000,317,51.459364,24.896430,...,1.172879,9.278142e-09,-0.377510,0.011382,13.394977,1.529152,2.000000,1.352393,1.774919,0.000198
4,RV-4,True,69.503227,1.447038,-1375.321599,2.460177e+06,0.962170,194,244.341203,63.321765,...,1.132455,7.659728e-09,1.438938,-0.005549,24.537941,1.391391,1.398056,1.556672,2.239142,0.000231


### Keep only the relevant features

We only keep the features from the Cadence/Error and General categories, plus the target value.

In [4]:
ds4 = ds4_orig[KEEP_CS_FEATURES + TARGET_DS4].copy()
ds4.head()

,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,all_times_nhist_peak_2_to_4,...,qso_log_chi2_qsonu,qso_log_chi2nuNULL_chi2nu,skew,std,stetson_j,stetson_k,weighted_average,fold2P_slope_10percentile,fold2P_slope_90percentile,Pulsating
0,8.0,22.0,32.0,34.0,47.0,1.037736,1.122449,1.527778,1.081633,1.472222,...,11.309243,-0.256635,-0.781648,3.165073,24.980336,0.958442,640.080094,-804.937332,734.364181,True
1,4.0,25.0,23.0,49.0,NaN,1.500000,2.000000,NaN,1.333333,NaN,...,10.193226,-0.719675,-0.066318,2.172986,18.856633,1.041665,1393.651760,-0.000084,0.000069,True
2,7.0,9.0,16.0,26.0,43.0,1.500000,1.500000,1.615385,1.000000,1.076923,...,9.388395,0.509991,0.234427,2.803447,22.406281,0.974131,1798.934279,-0.000131,0.000092,False
3,9.0,10.0,46.0,49.0,0.0,1.000000,1.000000,2.000000,1.000000,2.000000,...,12.881028,-0.441638,-0.583404,3.021310,27.726903,0.968993,918.862056,-0.000110,0.000187,False
4,7.0,23.0,26.0,19.0,3.0,1.000000,1.500000,3.000000,1.500000,3.000000,...,9.928810,-0.441719,0.653280,2.835690,24.866941,1.020485,-1374.121335,-0.000017,0.000029,True


We convert the `Pulsating` label to 0 / 1 values:

In [5]:
ds4.Pulsating = ds4.Pulsating.astype(int)
ds4.head()

,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,all_times_nhist_peak_2_to_4,...,qso_log_chi2_qsonu,qso_log_chi2nuNULL_chi2nu,skew,std,stetson_j,stetson_k,weighted_average,fold2P_slope_10percentile,fold2P_slope_90percentile,Pulsating
0,8.0,22.0,32.0,34.0,47.0,1.037736,1.122449,1.527778,1.081633,1.472222,...,11.309243,-0.256635,-0.781648,3.165073,24.980336,0.958442,640.080094,-804.937332,734.364181,1
1,4.0,25.0,23.0,49.0,NaN,1.500000,2.000000,NaN,1.333333,NaN,...,10.193226,-0.719675,-0.066318,2.172986,18.856633,1.041665,1393.651760,-0.000084,0.000069,1
2,7.0,9.0,16.0,26.0,43.0,1.500000,1.500000,1.615385,1.000000,1.076923,...,9.388395,0.509991,0.234427,2.803447,22.406281,0.974131,1798.934279,-0.000131,0.000092,0
3,9.0,10.0,46.0,49.0,0.0,1.000000,1.000000,2.000000,1.000000,2.000000,...,12.881028,-0.441638,-0.583404,3.021310,27.726903,0.968993,918.862056,-0.000110,0.000187,0
4,7.0,23.0,26.0,19.0,3.0,1.000000,1.500000,3.000000,1.500000,3.000000,...,9.928810,-0.441719,0.653280,2.835690,24.866941,1.020485,-1374.121335,-0.000017,0.000029,1


Store the resulting dataframe:

In [6]:
ds4.to_csv(OUT_FOLDER + SYNTH_PROCESSED_DS, sep=',', decimal='.', index=False)

## Train/test set and validation set split

We do a train/test set and validation set split.

**NOTE:** it is important to stratify the split with the target variable (`Pulsating` column).

We first separate the features from the target variable:

In [7]:
X = ds4.drop(columns=TARGET_DS4).copy()
y = ds4[TARGET_DS4].copy()

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VALIDATION_SIZE,
                                                    stratify=y, random_state=RANDOM_STATE)

In [9]:
X_train.shape

(750, 67)

In [10]:
y_train.shape

(750, 1)

In [11]:
y_train[:10]

,Pulsating
303,0
425,1
6,1
457,1
619,0
34,1
935,0
52,1
674,1
33,1


In [12]:
np.ravel(y_train)[:10]

array([0, 1, 1, 1, 0, 1, 0, 1, 1, 1])

In [13]:
X_val.shape

(250, 67)

In [14]:
y_val.shape

(250, 1)

## Design the model for optimization

At the moment, we don't use the `Pipeline` for simplification. It would be better to do this inside a `Pipeline` in case more steps are needed to be introduced later (for example, introduce the transformation to drop certain columns, a `RobustScaler`, etc.).

For optimization and training of the model we use telescopic search (Grid search followed by Random search) and 5-fold cross-validation, with the `auc`metric.

## Optimize and train the model

### Telescopic search step 1: Grid Search CV

In [15]:
# Define the Grid Search model:
clf = GridSearchCV(estimator=HistGradientBoostingClassifier(random_state=RANDOM_STATE),
                   param_grid=PARAM_GRID, cv=5, scoring='roc_auc',
                   verbose=1)

#### Optimize / train

In [16]:
# DISABLE WARNINGS:
warnings.filterwarnings('ignore')

# Optimize / train:
clf.fit(X_train, np.ravel(y_train))

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

...

C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
C:\Users\ciroe\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, wher

GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(random_state=11),
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_bins': [32, 128, 255],
                         'max_depth': [None, 3, 12, 48],
                         'max_iter': [50, 100, 200],
                         'max_leaf_nodes': [None, 16, 31, 60],
                         'min_samples_leaf': [10, 20, 40]},
             scoring='roc_auc', verbose=1)

We now show the parameters of the estimator:

In [18]:
# Show the grid search parameters:
clf.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__categorical_features': None,
 'estimator__early_stopping': 'auto',
 'estimator__l2_regularization': 0.0,
 'estimator__learning_rate': 0.1,
 'estimator__loss': 'auto',
 'estimator__max_bins': 255,
 'estimator__max_depth': None,
 'estimator__max_iter': 100,
 'estimator__max_leaf_nodes': 31,
 'estimator__min_samples_leaf': 20,
 'estimator__monotonic_cst': None,
 'estimator__n_iter_no_change': 10,
 'estimator__random_state': 11,
 'estimator__scoring': 'loss',
 'estimator__tol': 1e-07,
 'estimator__validation_fraction': 0.1,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': HistGradientBoostingClassifier(random_state=11),
 'n_jobs': None,
 'param_grid': {'learning_rate': [0.05, 0.1, 0.2],
  'max_iter': [50, 100, 200],
  'max_leaf_nodes': [None, 16, 31, 60],
  'max_depth': [None, 3, 12, 48],
  'min_samples_leaf': [10, 20, 40],
  'max_bins': [32, 128, 255]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': F

#### Best params found

In [20]:
# Show the best parameters found:
clf.best_params_

{'learning_rate': 0.05,
 'max_bins': 255,
 'max_depth': None,
 'max_iter': 50,
 'max_leaf_nodes': None,
 'min_samples_leaf': 40}

#### Best estimator

In [21]:
clf.best_estimator_

HistGradientBoostingClassifier(learning_rate=0.05, max_iter=50,
                               max_leaf_nodes=None, min_samples_leaf=40,
                               random_state=11)

#### Best estimator performance on validation set (just as a reference)

In [26]:
# PRINT METRICS:
print("GridSearchCV - precision on validation set: %f"
      %precision_score(y_val, clf.best_estimator_.predict(X_val)))
print("GridSearchCV - accuracy on validation set: %f"
      %accuracy_score(y_val, clf.best_estimator_.predict(X_val)))
print("GridSearchCV - recall on validation set: %f"
      %recall_score(y_val, clf.best_estimator_.predict(X_val)))
print("GridSearchCV - F1 score on validation set: %f"
      %f1_score(y_val, clf.best_estimator_.predict(X_val)))
print("GridSearchCV - ROC - AUC on validation set: %f"
      %roc_auc_score(y_val, clf.best_estimator_.predict(X_val)))


GridSearchCV - precision on validation set: 0.693277
GridSearchCV - accuracy on validation set: 0.664000
GridSearchCV - recall on validation set: 0.937500
GridSearchCV - F1 score on validation set: 0.797101
GridSearchCV - ROC - AUC on validation set: 0.475507


Compared to the results of the Off-the-Shelf classifier, it improves on accuracy, recall, and F1 score, although there is a slight decrease in precision and in ROC-AUC.